## How to use SplitsTrees in command line mode and programatically

Java must be installed in the machine.

Get and install SplitsTrees in the console

```console
wget https://software-ab.informatik.uni-tuebingen.de/download/splitstree4/splitstree4_unix_4_16_1.sh
bash splitstree4_unix_4_16_1.sh 
```

In the last command a bunch of orders need to be followed

In order to run it in command line we need to create a virtual instance to avoid X11 errors, for that the manual recommends to use xvfb, neither of conda wrappers worked here.

```
sudo apt install xvfb
```

Splitstree in command line mode is controled by a command file with instructions. That file must contain something like:

```
begin SplitsTree;
LOAD FILE=./analysis-nanuq/MultipleABfig/_002_NANUQdist_alpha0.95_beta0.095.nex
UPDATE
EXPORTGRAPHICS FORMAT=JPG FILE=network.jpg
QUIT
end;
```

These instruction load the distances produced by NANUQ and compute the network which is exported in jpg or orther formats

Finally, SplitsTree must be executed with the following command:
```
xvfb-run --auto-servernum --server-num=1 bash splitstree4/SplitsTree --commandLineMode --commandFile command_File.txt 
```

<div class="alert alert-box alert-danger">
    If <code>xvfb</code> is not used some parts of the process fail, like: 

<code>splitstree4.core.SplitsException: EXPORTGRAPHICS: invalid network block (try 'update' first)</code>
</div>

<div class="alert alert-box alert-success">
Apparently those problems are caused because Splitstree needs a virtual instance (virtual Frame Buffer) to run. Using <code>xvfb-run</code> solves this problem.

## Reconstruct networks with SplitsTree using NANUQ output

In [ ]:
#Imports
import os

In [1]:
#Define folder
folder = "/home/camayal/viburnumThings/Viburnum-Oreinotinus/notebooks/Oct2020/analysis-nanuq/MultipleABfig_2/"

In [25]:
#Create folder networks_in_Splitstree if not exists 
if not os.path.exists(f"{folder}networks"):
    os.makedirs(f"{folder}networks")

#Get the list of nex files produced by nanuq
#create a temporary command file for splitstree and execute splitstree
files = os.listdir(folder)
for file in sorted(files):
    if file.endswith('.nex'):
        with open(f"{folder}commands_for_splitstree.tmp", "w") as file_w:
            L = ["begin SplitsTree;\n",
                 f"LOAD FILE={folder}{file}\n",
                 "UPDATE\n",
                 f"EXPORTGRAPHICS FORMAT=JPG FILE={folder}networks/{file[:-4]}_network.jpg REPLACE=yes\n",
                 "QUIT\n",
                 "end;\n"]
            file_w.writelines(L)
        #Run splitstree for each nexus file produced by NANUQ
        !xvfb-run --auto-servernum --server-num=1 bash ~/splitstree4/SplitsTree --commandLineMode --silentMode --commandFile {folder}commands_for_splitstree.tmp 

## Construct concatenated figures with simplex and network

For this and video section it is mandatory to use imagemagick.

Install ImageMagick and dependencies for video creation:

```
conda install -c conda-forge ffmpeg -y
conda install -c conda-forge imagemagick -y
```

In [73]:
#Get every simplex and put in a list
#search the same filename in Network folder and concatenate both images
#save it in composed folder

if not os.path.exists(f"{folder}composed"):
    os.makedirs(f"{folder}composed")

files = os.listdir(f"{folder}")

for file in sorted(files):
    if file.endswith('.nex'):
        #Execute  ImageMagick  to concatenate images
        !convert \
        {folder}simplex/{file[:-4]}.jpg \
        {folder}networks/{file[:-4]}_network.jpg \
        +append \
        -resize 90% \
        {folder}composed/{file[:-4]}_simplex-plus-network.jpg
        
        
        

## Compose the video with concatenated figures

In [74]:
#Imports
from IPython.display import Video

In [75]:
paths = folder.split("/")
parent_folder = "/".join(paths[:-2])
name_video = paths[-2]

#Execute  ImageMagick  to create the final video
!convert \
-delay 20 \
-loop 0 \
{folder}composed/*.jpg \
{parent_folder}/{name_video}.mp4

print(f"{parent_folder}/{name_video}.mp4")

/home/camayal/viburnumThings/Viburnum-Oreinotinus/notebooks/Oct2020/analysis-nanuq/MultipleABfig_2.mp4


In [77]:
Video("analysis-nanuq/MultipleABfig_2.mp4", height=450)

In [33]:
#Old video with networks reconstructed with Phargorn
#Video("analysis-nanuq/nanuqTests_MultipleABfig_onlyAlpha_conct.mp4", width=950)